In [4]:
import hail as hl

In [5]:
hl.init()

Running on Apache Spark version 2.2.0
SparkUI available at http://10.1.7.107:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version devel-3e72697c3f47
NOTE: This is a beta version. Interfaces may change
  during the beta period. We recommend pulling
  the latest changes weekly.


In [6]:
schema = {'variant': hl.tstr, 'rsid': hl.tstr, 'nCompleteSamples': hl.tint32,
         'AC': hl.tint32, 'ytx': hl.tfloat64, 'beta': hl.tfloat64,
         'se': hl.tfloat64, 'tstat': hl.tfloat64, 'pval': hl.tfloat64}

standing_height = hl.import_table('/Users/maccum/manhattan/50.assoc.tsv', key='variant')

2018-06-26 14:31:44 Hail: INFO: Reading table with no type imputation
  Loading column 'variant' as type 'str' (type not specified)
  Loading column 'rsid' as type 'str' (type not specified)
  Loading column 'nCompleteSamples' as type 'str' (type not specified)
  Loading column 'AC' as type 'str' (type not specified)
  Loading column 'ytx' as type 'str' (type not specified)
  Loading column 'beta' as type 'str' (type not specified)
  Loading column 'se' as type 'str' (type not specified)
  Loading column 'tstat' as type 'str' (type not specified)
  Loading column 'pval' as type 'str' (type not specified)



In [6]:
sh_path = '/Users/maccum/manhattan/standing_height/'

In [ ]:
standing_height.write(sh_path, overwrite=True)

In [7]:
standing_height = hl.read_table(sh_path)

In [8]:
standing_height.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'variant': str 
    'rsid': str 
    'nCompleteSamples': str 
    'AC': str 
    'ytx': str 
    'beta': str 
    'se': str 
    'tstat': str 
    'pval': str 
----------------------------------------
Key: ['variant']
----------------------------------------


In [9]:
standing_height.count()

10894596

In [10]:
import generator as gen

<module 'generator' from '/Users/maccum/anaconda3/lib/python3.6/site-packages/generator/__init__.py'>